In [ ]:
cd ..

In [ ]:
import json
from importlib import reload
import km.orchestrator.orchestrator as ork
import numpy as np

In [ ]:
serialized_model_path = "serialized_models/lda_model.pkl"
orchestrator = ork.Orchestrator()
model = orchestrator.load_model(serialized_model_path)
model.explain()

In [ ]:
documents = orchestrator._get_documents()

In [ ]:
query_document = orchestrator._get_documents(3)[-1]
print(query_document)
query = query_document.content
top_results = orchestrator.query_documents(query)

In [ ]:
top_users = orchestrator.query_users(query)

# Smarter labeling of users

In [ ]:
!curl -o notebooks/names.csv https://raw.githubusercontent.com/hadley/data-baby-names/master/baby-names.csv

In [ ]:
import json
import pandas as pd
from sklearn.cluster import KMeans

In [ ]:
kmeans = KMeans(25, n_init=8)

In [ ]:
X = [doc.representation for doc in documents]
labels = kmeans.fit_predict(X)
docs_with_labels = list(zip(documents, labels))

In [ ]:
names = pd.read_csv("notebooks/names.csv")
names = list(set(names.name))

In [ ]:
users_dict = {}
for i in range(60):
    clusters = np.random.choice(range(25), 4, replace=False)
    user_docs = []
    for cluster in clusters:
        user_docs.extend([d.id for d, l in docs_with_labels if l == cluster])
    users_dict[i] = {"email": f"{names[i].lower()}@yourcompany.com", "document_ids": user_docs}

In [ ]:
json.dump(users_dict, open("../data/news-articles/user_labels.json", "w"), indent=4, sort_keys=True)